In [2]:
%matplotlib inline


import pandas as pd# data analysis
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn



DATA_DIR = '/home/msingh/Documents/pair_projects/'

In [9]:
from subprocess import check_output 
print(check_output(["ls", "/home/msingh/Documents/pair_projects/"]).decode("utf8")) 

# Any results you write to the current directory are saved as output. 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.cross_validation import train_test_split 
from sklearn import metrics 
from sklearn.cross_validation import KFold, cross_val_score 
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import roc_auc_score 
from collections import defaultdict 
%pylab inline 
pylab.rcParams['figure.figsize'] = (10, 6)

pair_projects.ipynb
sample_submission.csv
test_ver2.csv
train_ver2.csv

Populating the interactive namespace from numpy and matplotlib


In [10]:
# Calibrate the number of rows to not crash the kernel 
limit_rows = 50000 
df = pd.read_csv("train_ver2.csv",dtype={"sexo":str, "ind_nuevo":str, "ult_fec_cli_1t":str, "indext":str}, nrows=limit_rows) 
test = pd.read_csv("test_ver2.csv",dtype={"sexo":str, "ind_nuevo":str, "ult_fec_cli_1t":str, "indext":str}, nrows=limit_rows) 
# Format to datetime data,maybe the month of the year is important to purchase 

df["month"] = pd.DatetimeIndex(df["fecha_dato"]).month 
df["age"] = pd.to_numeric(df["age"], errors="coerce") 

# Not very sure just to keep with unique_ids 'cause principal id is fecha_dato + ncodpers and if we 
# eliminate repeated ncodpers we lose some important inf

/home/msingh/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/msingh/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# eliminate repeated ncodpers we lose some important information. 
#We take unique_ids and unique fecha_datos just to play forward. 
unique_ids = pd.Series(df["ncodpers"].unique()) 
unique_fecha_dato = df["fecha_dato"].unique() 
df.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,month
0,1/28/2015,1050622,N,ES,H,23.0,8/10/2012,0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
1,1/28/2015,1050492,N,ES,V,25.0,8/10/2012,0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
2,1/28/2015,1050571,N,ES,H,23.0,8/10/2012,0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
3,1/28/2015,1050915,N,ES,H,37.0,8/10/2012,0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1
4,1/28/2015,1050732,N,ES,H,23.0,8/10/2012,0,35,1.0,...,0,0,0,0,0,0,0.0,0.0,0,1


In [12]:
train_unique_customers = set(df.ncodpers.unique()) 
test_unique_customers = set(test.ncodpers.unique()) 
print("Number of customers in train : ", len(train_unique_customers)) 
print("Number of customers in test : ", len(test_unique_customers)) 
print("Number of common customers : ", len(train_unique_customers.intersection(test_unique_customers)))

('Number of customers in train : ', 48695)
('Number of customers in test : ', 38171)
('Number of common customers : ', 2237)


In [13]:
# Change datatype 
df["age"] = pd.to_numeric(df["age"], errors="coerce") 
df["antiguedad"] = pd.to_numeric(df["antiguedad"], errors="coerce") 
df["indrel_1mes"] = pd.to_numeric(df["indrel_1mes"], errors="coerce") 

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 49 columns):
fecha_dato               50000 non-null object
ncodpers                 50000 non-null int64
ind_empleado             49891 non-null object
pais_residencia          49891 non-null object
sexo                     49891 non-null object
age                      49891 non-null float64
fecha_alta               49891 non-null object
ind_nuevo                49891 non-null object
antiguedad               49891 non-null float64
indrel                   49891 non-null float64
ult_fec_cli_1t           90 non-null object
indrel_1mes              49452 non-null float64
tiprel_1mes              49456 non-null object
indresi                  49891 non-null object
indext                   49891 non-null object
conyuemp                 5 non-null object
canal_entrada            49317 non-null object
indfall                  49891 non-null object
tipodom                  49891 non-null float64
c

In [15]:
df.isnull().sum()

fecha_dato                   0
ncodpers                     0
ind_empleado               109
pais_residencia            109
sexo                       109
age                        109
fecha_alta                 109
ind_nuevo                  109
antiguedad                 109
indrel                     109
ult_fec_cli_1t           49910
indrel_1mes                548
tiprel_1mes                544
indresi                    109
indext                     109
conyuemp                 49995
canal_entrada              683
indfall                    109
tipodom                    109
cod_prov                   340
nomprov                    340
ind_actividad_cliente      109
renta                    10375
segmento                   694
ind_ahor_fin_ult1            0
ind_aval_fin_ult1            0
ind_cco_fin_ult1             0
ind_cder_fin_ult1            0
ind_cno_fin_ult1             0
ind_ctju_fin_ult1            0
ind_ctma_fin_ult1            0
ind_ctop_fin_ult1            0
ind_ctpp

In [16]:
# Drop the columns with majority of missing values 
df = df.drop(["ult_fec_cli_1t", "conyuemp"], axis=1) 

In [17]:
df.isnull().sum()

fecha_dato                   0
ncodpers                     0
ind_empleado               109
pais_residencia            109
sexo                       109
age                        109
fecha_alta                 109
ind_nuevo                  109
antiguedad                 109
indrel                     109
indrel_1mes                548
tiprel_1mes                544
indresi                    109
indext                     109
canal_entrada              683
indfall                    109
tipodom                    109
cod_prov                   340
nomprov                    340
ind_actividad_cliente      109
renta                    10375
segmento                   694
ind_ahor_fin_ult1            0
ind_aval_fin_ult1            0
ind_cco_fin_ult1             0
ind_cder_fin_ult1            0
ind_cno_fin_ult1             0
ind_ctju_fin_ult1            0
ind_ctma_fin_ult1            0
ind_ctop_fin_ult1            0
ind_ctpp_fin_ult1            0
ind_deco_fin_ult1            0
ind_deme